# Example of Model Deployment

https://dataplatform.ibm.com/docs/content/analyze-data/ml-deploy.html

In [ ]:
import pandas as pd
import numpy as np

### Import `madrid_train.csv` 

In [ ]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_01fa29576c1949bf8bf19f9e93a1ba4a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='sEmL__NZBUKJEpzPUIPnf4zqGziaw9NmQpRqVC81YaQ_',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_01fa29576c1949bf8bf19f9e93a1ba4a.get_object(Bucket='watsondeploymentexampleminimalwit-donotdelete-pr-ufwoo44tqds7eb',Key='madrid_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()



In [ ]:
houses_raw = df_data_3
houses_raw.head()

# Filter Data

In [ ]:
filterd_on_price = houses_raw[ houses_raw["price"] > 50000 ]
filtered_on_area = filterd_on_price[filterd_on_price["mts2"] > 10 ]
houses = filtered_on_area
houses.head()


## XGBoost within a pipeline - XGBoost is 1 of the supported models

In [ ]:
import xgboost

In [ ]:
filtered_on_area.columns

In [ ]:
houses_feature_engineered = filtered_on_area

In [ ]:
X = houses_feature_engineered[ ['mts2', 'rooms', 'distance_to_centre', 'sauna'] ]
y = houses_feature_engineered[ ["price"] ]

In [ ]:
from sklearn.cross_validation import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [ ]:
from xgboost.sklearn import XGBClassifier, XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score, mean_absolute_error

from xgboost import plot_importance
from matplotlib import pyplot

import pprint

In [ ]:
pipeline = Pipeline([('scaler', StandardScaler()), ('regressor', XGBRegressor(n_estimators=10, learning_rate=0.2))])
pipeline

In [ ]:
pipeline.fit(X_train.values, y_train.values)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

labels_of_columns = list(X.columns)
xgbooster_of_fit = pipeline.steps[1][1].booster()
feature_scores = xgbooster_of_fit.get_fscore()
labels,feat_importance = zip(*[ (labels_of_columns[int(k[1:])],imp) for (k,imp) in feature_scores.items() ])

fig, ax = plt.subplots()

y_ticks = np.array(range(len(feat_importance)))
width=.8

ax.barh(y_ticks, feat_importance, color="blue")
ax.set_yticklabels(labels)
ax.set_yticks(y_ticks + width / 2)
plt.show()

## Store the Model with the Watson Machine Learning Service

Example notebook

https://dataplatform.ibm.com/exchange/public/entry/view/30b8df0ef74241a0516f9e81cd6d7029

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## Change the following cell with your own credentials of Watson Machine Learning serive

In [ ]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "JTWUrvdQerqXu3Y/Fcf0wBoxs1eoUpp3ivXynFqbNIBZN3amVmF80kroVV8Cpce9HxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "9fb26c5d-919f-491b-a47e-172479d63b4c",
  "password": "3527ddba-55ef-4039-a039-b8135aabc66f",
  "instance_id": "3423aa28-09e1-4287-985a-1e77f6018a5f"
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props = {client.repository.ModelMetaNames.NAME: "XGBoost for madrid house prices"}
model_details = client.repository.store_model(pipeline, model_props)

In [ ]:
# Display a list of all the models.
client.repository.list_models()

In [ ]:
# Extract the uid.
model_uid = client.repository.get_model_uid(model_details)
print(model_uid)

In [ ]:
# Create the deployment.
deployment_details = client.deployments.create(model_uid, 'XGBoost for madrid house prices')

In [ ]:
# List the deployments.
client.deployments.list()

In [ ]:
# Extract endpoint url and display it.
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

In [ ]:
# Prepare scoring payload.
payload_scoring = {"values": [[100,2,0.1,0]]}
print(payload_scoring)

In [ ]:
# Perform prediction and display the result.
response_scoring = client.deployments.score(scoring_url, payload_scoring)
print(response_scoring)

In [ ]:
client.repository.list_models()

## Check that under Assets > Models your model is now saved. 

Open your model, and check out `Overview` `Evaluation` and `Deployments`. Not all Models have deployments, but the upper code made sure we have a deployment.

Under `Deployments > Implementation` you can find common code snippets. We are going to use the python snippet to create a simple webapp.